In [137]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path  
import os
import math
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

In [4]:
data = pd.read_excel('500_stocks_data.xlsx', sheet_name = 'price_data')

In [6]:
data1 = data[2:]
data1.columns = data1.iloc[0]
data1 = data1[1:]

In [49]:
column_dates = data1.iloc[1:, 0]

In [92]:
for i in range(0, 4023, 8):
    temp = data1.iloc[:, i: i+7]
    temp_df_name = temp.columns[0].split()[0]
    temp.columns = temp.iloc[0]
    temp = temp[1:]
    temp['Dates'] = column_dates
    temp = temp.reset_index(drop = True)
    temp = temp.set_index('Dates')
    temp.columns.name = np.nan
    filepath = Path('/Users/steveszeng/Desktop/Equity_StatArb-master/data/' + temp_df_name + '.csv')
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    temp.to_csv(filepath) 

/Users/steveszeng/opt/anaconda3/envs/mfe230e/lib/python3.11/site-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


In [170]:
folder_path = '/Users/steveszeng/Desktop/Equity_StatArb-master/data/'

# get all the file names
files = os.listdir(folder_path)

# read all the files 
price = pd.DataFrame()
for file in files:
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        df = df[['Dates', 'PX_LAST']]
        ticker = file.split('.')[0]
        df = df.rename(columns = {'PX_LAST': ticker})
        if price.empty:
            price = df
        else:
            price = pd.merge(price, df, on = 'Dates')

price['Dates'] = pd.to_datetime(price['Dates'])
price = price.set_index('Dates')

In [171]:
returns = price.pct_change()

In [133]:
np.sum(np.isnan(returns), axis = 1)

Dates
2001-01-01    503
2001-01-02    142
2001-01-03    142
2001-01-04    142
2001-01-05    142
             ... 
2024-06-20      2
2024-06-21      2
2024-06-24      2
2024-06-25      2
2024-06-26      2
Length: 6128, dtype: int64

In [172]:
returns = returns.fillna(0)

In [173]:
returns

,CSCO,UAL,TROW,ISRG,NVR,TPR,DVN,CE,MRO,BA,...,WM,DOV,CRM,PGR,WAT,IEX,BWA,LRCX,BLK,PPL
Dates,,,,,,,,,,,,,,,,,,,,,
2001-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2001-01-02,-0.129085,0.000000,-0.047689,-0.040233,-0.056634,-0.091324,-0.032311,0.000000,0.006755,-0.060606,...,-0.027027,-0.047765,0.000000,-0.038597,-0.055389,-0.049055,-0.006253,-0.008621,-0.047619,-0.041497
2001-01-03,0.240150,0.000000,0.055901,0.072256,0.025729,-0.002389,-0.046441,0.000000,-0.022371,0.009081,...,0.055556,0.074432,0.000000,-0.021958,-0.055468,0.047621,0.067612,0.200000,-0.004675,-0.053390
2001-01-04,0.013616,0.000000,-0.038235,-0.092593,0.011706,-0.040764,-0.043548,0.000000,0.004572,-0.024983,...,-0.043860,0.003012,0.000000,-0.038486,-0.060403,0.024620,0.014745,0.054348,-0.011001,-0.121949
2001-01-05,-0.125373,0.000000,0.009174,-0.023810,0.028926,-0.024992,0.076008,0.000000,0.029623,-0.036885,...,0.000000,-0.027024,0.000000,-0.040693,-0.048214,-0.057300,-0.042093,0.006873,-0.092063,0.032982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-20,0.016315,-0.013715,0.005273,-0.005632,-0.003300,0.007315,0.007448,-0.007763,0.018668,0.007486,...,0.004233,0.002790,0.043096,0.000474,-0.014946,0.004052,0.003635,-0.025946,-0.001046,0.000357
2024-06-21,0.012200,-0.014724,0.001946,0.000023,0.014843,0.007988,-0.003696,-0.007606,-0.002515,0.001475,...,0.001054,-0.004364,0.013482,-0.007873,0.000931,-0.002313,0.000000,-0.010327,0.006093,-0.003209
2024-06-24,-0.000211,0.020756,0.003884,-0.008138,0.014995,0.026177,0.039721,0.014964,0.035303,0.014386,...,0.016508,0.010411,-0.020893,0.007123,-0.003445,0.008880,0.014187,-0.024687,0.014411,0.007511


In [167]:
columns 

Index(['CSCO', 'UAL', 'TROW', 'ISRG', 'NVR', 'TPR', 'DVN', 'CE', 'MRO', 'BA',
       ...
       'WM', 'DOV', 'CRM', 'PGR', 'WAT', 'IEX', 'BWA', 'LRCX', 'BLK', 'PPL'],
      dtype='object', length=501)

In [169]:
set(returns.columns) - set(columns)

{'BF', 'BRK'}